# Persiapan Data dan Environment

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report
import pickle
import os

In [9]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

split_idx = 40000
x_train = x_train_full[:split_idx]
y_train = y_train_full[:split_idx]
x_val = x_train_full[split_idx:]
y_val = y_train_full[split_idx:]

y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()

print(f"Training data: {x_train.shape}, {y_train.shape}")
print(f"Validation data: {x_val.shape}, {y_val.shape}")
print(f"Test data: {x_test.shape}, {y_test.shape}")

Training data: (40000, 32, 32, 3), (40000,)
Validation data: (10000, 32, 32, 3), (10000,)
Test data: (10000, 32, 32, 3), (10000,)


# Implementasi Model CNN Baseline

In [5]:
def create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], 
                     kernel_sizes=[3, 3], pooling_type='max'):
    model = keras.Sequential()
    
    model.add(layers.Input(shape=(32, 32, 3)))
    
    for i in range(conv_layers):
        filters = filters_per_layer[i] if i < len(filters_per_layer) else filters_per_layer[-1]
        kernel_size = kernel_sizes[i] if i < len(kernel_sizes) else kernel_sizes[-1]
        
        model.add(layers.Conv2D(filters, kernel_size, activation='relu', padding='same'))
        
        if pooling_type == 'max':
            model.add(layers.MaxPooling2D(2, 2))
        elif pooling_type == 'avg':
            model.add(layers.AveragePooling2D(2, 2))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

In [10]:
def train_model(model, model_name, epochs=2):
  model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=epochs,
    validation_data=(x_val, y_val),
    verbose=1
  )

  model.save(f'models/{model_name}.h5')

  with open(f'histories/{model_name}_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
  
  return model, history

# Eksperimen Hyperparameter

In [11]:
# Pengaruh Jumlah Layer Konvolusi
os.makedirs('models', exist_ok=True)
os.makedirs('histories', exist_ok=True)

model_1_layer = create_cnn_model(conv_layers=1, filters_per_layer=[32])
model_1_layer, history_1_layer = train_model(model_1_layer, 'model_1_conv_layer')

model_2_layer = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64])
model_2_layer, history_2_layer = train_model(model_2_layer, 'model_2_conv_layers')

model_3_layer = create_cnn_model(conv_layers=3, filters_per_layer=[32, 64, 128])
model_3_layer, history_3_layer = train_model(model_3_layer ,'model_3_conv_layers')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.2361 - loss: 2.0307 - val_accuracy: 0.4756 - val_loss: 1.5427
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.3579 - loss: 1.6852 - val_accuracy: 0.4956 - val_loss: 1.4451


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.3134 - loss: 1.8554 - val_accuracy: 0.5326 - val_loss: 1.3111
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.5023 - loss: 1.3788 - val_accuracy: 0.5992 - val_loss: 1.1314


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.2969 - loss: 1.8897 - val_accuracy: 0.5483 - val_loss: 1.2481
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.5247 - loss: 1.3250 - val_accuracy: 0.6210 - val_loss: 1.0710


In [12]:
# Pengaruh Jumlah Filter per Layer
model_low_filters = create_cnn_model(conv_layers=2, filters_per_layer=[16, 32])
model_low_filters, history_low_layer = train_model(model_low_filters, 'model_low_filters')

model_med_filters = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64])
model_med_filters, history_med_layer = train_model(model_med_filters, 'model_med_filters')

model_high_filters = create_cnn_model(conv_layers=2, filters_per_layer=[64, 128])
model_high_filters, history_high_layer = train_model(model_high_filters, 'model_high_filters')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.3100 - loss: 1.8757 - val_accuracy: 0.5096 - val_loss: 1.3667
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5012 - loss: 1.3843 - val_accuracy: 0.5978 - val_loss: 1.1517


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.3071 - loss: 1.8645 - val_accuracy: 0.5350 - val_loss: 1.3173
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.5032 - loss: 1.3660 - val_accuracy: 0.5919 - val_loss: 1.1387


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 46s 36ms/step - accuracy: 0.3233 - loss: 1.8296 - val_accuracy: 0.5639 - val_loss: 1.2570
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 46s 37ms/step - accuracy: 0.5260 - loss: 1.3281 - val_accuracy: 0.6156 - val_loss: 1.1481


In [13]:
# Pengaruh Ukuran Filter
model_small_kernel = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], kernel_sizes=[3, 3])
model_small_kernel, history_small_kernel = train_model(model_small_kernel, 'model_small_kernel')

model_med_kernel = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], kernel_sizes=[5, 5])
model_med_kernel, history_med_kernel = train_model(model_med_kernel, 'model_med_kernel')

model_large_kernel = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], kernel_sizes=[7, 7])
model_large_kernel, history_large_kernel = train_model(model_large_kernel, 'model_large_kernel')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 27s 20ms/step - accuracy: 0.3217 - loss: 1.8271 - val_accuracy: 0.5478 - val_loss: 1.2705
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.5076 - loss: 1.3690 - val_accuracy: 0.5966 - val_loss: 1.1283


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.3092 - loss: 1.8705 - val_accuracy: 0.5236 - val_loss: 1.3296
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.4953 - loss: 1.3963 - val_accuracy: 0.5839 - val_loss: 1.1769


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - accuracy: 0.2856 - loss: 1.9302 - val_accuracy: 0.4828 - val_loss: 1.4571
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 33s 26ms/step - accuracy: 0.4739 - loss: 1.4761 - val_accuracy: 0.5504 - val_loss: 1.2655


In [14]:
# Pengaruh Jenis Pooling
model_max_pool = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], pooling_type='max')
model_max_pool, history_max_pool = train_model(model_max_pool, 'model_max_pooling')

model_avg_pool = create_cnn_model(conv_layers=2, filters_per_layer=[32, 64], pooling_type='avg')
model_avg_pool, history_avg_pool = train_model(model_avg_pool, 'model_avg_pooling')

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 27s 20ms/step - accuracy: 0.3141 - loss: 1.8619 - val_accuracy: 0.5286 - val_loss: 1.3242
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 20ms/step - accuracy: 0.5081 - loss: 1.3701 - val_accuracy: 0.5896 - val_loss: 1.1648


Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 27s 19ms/step - accuracy: 0.3171 - loss: 1.8513 - val_accuracy: 0.5226 - val_loss: 1.3594
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.4934 - loss: 1.4008 - val_accuracy: 0.5728 - val_loss: 1.2134


# Evaluasi dan Analisis

In [1]:
def evaluate_model(model, model_name):
  y_pred_proba = model.predict(x_test)
  y_pred = np.argmax(y_pred_proba, axis=1)

  f1_macro = f1_score(y_test, y_pred, average='macro')

  test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

  print(f"Model: {model_name}")
  print(f"Test Accuracy: {test_acc:.4f}")
  print(f"Macro F1-Score: {f1_macro:.4f}")
  print("-" * 40)

  return test_acc, f1_macro


In [3]:
def plot_training_history(histories, labels, title):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    for history, label in zip(histories, labels):
        plt.plot(history['loss'], label=f'{label} - Training')
        plt.plot(history['val_loss'], label=f'{label} - Validation', linestyle='--')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    for history, label in zip(histories, labels):
        plt.plot(history['accuracy'], label=f'{label} - Training')
        plt.plot(history['val_accuracy'], label=f'{label} - Validation', linestyle='--')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Implementasi Forward Propagation From Scratch

In [4]:
from model import CNNFromScratch

# Testing dan Validasi

In [5]:
def test_forward_propagation(keras_model, model_name):
    """
    Test implementasi forward propagation from scratch
    """
    # Load model dari file
    keras_model = keras.models.load_model(f'models/{model_name}.h5')
    
    # Buat model from scratch
    scratch_model = CNNFromScratch(keras_model)
    
    # Test dengan subset data test (untuk efisiensi)
    test_samples = x_test[:100]
    
    # Prediksi dengan Keras
    keras_pred = keras_model.predict(test_samples)
    keras_pred_classes = np.argmax(keras_pred, axis=1)
    
    # Prediksi dengan implementasi from scratch
    scratch_pred = scratch_model.predict(test_samples)
    scratch_pred_classes = np.argmax(scratch_pred, axis=1)
    
    # Bandingkan hasil
    accuracy_match = np.mean(keras_pred_classes == scratch_pred_classes)
    
    # Hitung F1-score untuk kedua implementasi
    y_true = y_test[:100]
    keras_f1 = f1_score(y_true, keras_pred_classes, average='macro')
    scratch_f1 = f1_score(y_true, scratch_pred_classes, average='macro')
    
    print(f"Model: {model_name}")
    print(f"Prediction Match Accuracy: {accuracy_match:.4f}")
    print(f"Keras F1-Score: {keras_f1:.4f}")
    print(f"From Scratch F1-Score: {scratch_f1:.4f}")
    print(f"Difference in F1-Score: {abs(keras_f1 - scratch_f1):.6f}")
    print("-" * 50)
    
    return accuracy_match, keras_f1, scratch_f1

# Langkah Eksekusi Lengkap

In [10]:
def main():
    # 1. Persiapan data
    print("1. Loading and preprocessing CIFAR-10 data...")
    # (kode preprocessing di atas)
    
    # 2. Training semua variasi model
    print("2. Training models with different configurations...")
    # (kode training di atas)
    
    # 3. Evaluasi semua model
    print("3. Evaluating all models...")
    results = {}
    
    # Evaluasi untuk setiap eksperimen
    model_configs = [
        ('model_1_conv_layer', '1 Conv Layer'),
        ('model_2_conv_layers', '2 Conv Layers'),
        ('model_3_conv_layers', '3 Conv Layers'),
        ('model_low_filters', 'Low Filters'),
        ('model_med_filters', 'Medium Filters'),
        ('model_high_filters', 'High Filters'),
        ('model_small_kernel', 'Small Kernel'),
        ('model_med_kernel', 'Medium Kernel'),
        ('model_large_kernel', 'Large Kernel'),
        ('model_max_pooling', 'Max Pooling'),
        ('model_avg_pooling', 'Average Pooling')
    ]
    
    for model_name, display_name in model_configs:
        model = keras.models.load_model(f'models/{model_name}.h5')
        test_acc, f1_macro = evaluate_model(model, display_name)
        results[model_name] = {'accuracy': test_acc, 'f1_score': f1_macro}
    
    # 4. Plot perbandingan
    print("4. Plotting comparisons...")
    # (kode plotting di atas)
    
    # 5. Test forward propagation
    print("5. Testing forward propagation implementation...")
    for model_name, display_name in model_configs:
        test_forward_propagation(keras.models.load_model(f'models/{model_name}.h5'), model_name)
    
    # 6. Simpan hasil
    print("6. Saving results...")
    with open('experiment_results.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    print("All experiments completed!")

if __name__ == "__main__":
    main()

1. Loading and preprocessing CIFAR-10 data...
2. Training models with different configurations...
3. Evaluating all models...
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


Model: 1 Conv Layer
Test Accuracy: 0.5015
Macro F1-Score: 0.4974
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


Model: 2 Conv Layers
Test Accuracy: 0.5999
Macro F1-Score: 0.5983
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


Model: 3 Conv Layers
Test Accuracy: 0.6239
Macro F1-Score: 0.6126
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Model: Low Filters
Test Accuracy: 0.6007
Macro F1-Score: 0.5898
----------------------------------------
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


...

Model: model_small_kernel
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6034
From Scratch F1-Score: 0.6034
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


Model: model_med_kernel
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6724
From Scratch F1-Score: 0.6724
Difference in F1-Score: 0.000000
--------------------------------------------------


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


Model: model_large_kernel
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6731
From Scratch F1-Score: 0.6731
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


Model: model_max_pooling
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.6417
From Scratch F1-Score: 0.6417
Difference in F1-Score: 0.000000
--------------------------------------------------
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Model: model_avg_pooling
Prediction Match Accuracy: 1.0000
Keras F1-Score: 0.5543
From Scratch F1-Score: 0.5543
Difference in F1-Score: 0.000000
--------------------------------------------------
6. Saving results...
All experiments completed!
